In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import pydicom as dcm
import os
import cv2
import gc
import glob
from tqdm import tqdm
from matplotlib.patches import Rectangle
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.mobilenet import preprocess_input

In [ ]:
labels = pd.read_csv('../input/rsna-pneumonia-detection-challenge/stage_2_train_labels.csv')
details = pd.read_csv('../input/rsna-pneumonia-detection-challenge/stage_2_detailed_class_info.csv')

In [ ]:
# duplicates in details just have the same class so can be safely dropped
details = details.drop_duplicates('patientId').reset_index(drop=True)
labels_w_class = labels.merge(details, how='inner', on='patientId')

In [ ]:
labels_w_class.head()

### Dealing with missing values

In [ ]:
labels_w_class.info()
# No null values in patientId ,Target and Class

In [ ]:
# null values in x, y, width, height indicates that there is no pneumonia. Replacing null with 0
labels_w_class.fillna(0, inplace=True)
labels_w_class.info()

In [ ]:
new_df = labels_w_class.head(6000)
new_df.head()

In [ ]:
class_train, class_val = train_test_split(new_df, test_size=0.20, random_state=42, stratify=new_df['class'])

In [ ]:
IMAGE_SIZE = 1024
ADJUSTED_IMAGE_SIZE=224
MASK_IMAGE_SIZE = 28
FACTOR = MASK_IMAGE_SIZE/IMAGE_SIZE

In [ ]:
X_feature_tr = []
y_feature_target_tr = []
y_feature_coordinates_tr = []
from PIL import Image
train_images_dir = '../input/rsna-pneumonia-detection-challenge/stage_2_train_images/'
def create_mask(datafm):
    X = []
    y=[]
    masks = np.zeros((int(datafm.shape[0]), MASK_IMAGE_SIZE, MASK_IMAGE_SIZE))
    for index, patient_id in enumerate(datafm['patientId'].T.to_dict().values()):
        image_path = train_images_dir+patient_id+".dcm"
        img = dcm.read_file(image_path)
        img = img.pixel_array
        img = cv2.resize(img, (ADJUSTED_IMAGE_SIZE, ADJUSTED_IMAGE_SIZE), interpolation=cv2.INTER_NEAREST)
        img = Image.fromarray(img)
        img = img.convert('RGB')
        img = preprocess_input(np.array(img, dtype=np.float32))
        X.append(img)
        rows = labels_w_class[labels_w_class['patientId']==patient_id]
        y.append(rows['Target'].values[0])

        row_data = list(rows.T.to_dict().values())
        for row in row_data:
            x1 = int(row['x']*FACTOR)
            x2 = int((row['x']*FACTOR)+(row['width']*FACTOR))
            y1 = int(row['y']*FACTOR)
            y2 = int((row['y']*FACTOR)+(row['height']*FACTOR))
            masks[index][y1:y2, x1:x2] = 1
    del img,row,row_data
    gc.collect()
    X=np.array(X)
    y=np.array(y)
    return X, y, masks

In [ ]:
X_train, y_tr_target, y_train = create_mask(class_train)
X_val, y_val_target, y_val = create_mask(class_val)

In [ ]:
#Model I : U-Net
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.layers import Concatenate, Conv2D, Reshape, UpSampling2D
from tensorflow.keras.models import Model, load_model
import tensorflow as tf
from tensorflow import keras

In [ ]:
#this function will creat U-net model
def create_model(trainable=True):
    model = MobileNet(input_shape=(224, 224, 3), include_top=False, weights="imagenet")

    for layer in model.layers:
        layer.trainable = trainable

    block1 = model.get_layer("conv_pw_5_relu").output
    block2 = model.get_layer("conv_pw_11_relu").output
    block3 = model.get_layer("conv_pw_13_relu").output

    x = Concatenate()([UpSampling2D()(block3), block2])
    x = Concatenate()([UpSampling2D()(x), block1])

    x = Conv2D(1, kernel_size=1, activation="sigmoid")(x)
    x = Reshape((28, 28))(x)

    return Model(inputs=model.input, outputs=x)

In [ ]:
model = create_model(False)
#model.summary()

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

In [ ]:
# define iou or jaccard loss function
def iou_loss(y_true, y_pred):
    y_true = tf.reshape(y_true, [-1])
    y_pred = tf.reshape(y_pred, [-1])
    intersection = tf.reduce_sum(y_true * y_pred)
    score = (intersection + 1.) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection + 1.)
    return 1 - score

# combine bce loss and iou loss
def iou_bce_loss(y_true, y_pred):
    return 0.5 * keras.losses.binary_crossentropy(y_true, y_pred) + 0.5 * iou_loss(y_true, y_pred)

# mean iou as a metric
def mean_iou(y_true, y_pred):
    y_pred = tf.round(y_pred)
    intersect = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred)
    smooth = tf.ones(tf.shape(intersect))
    return tf.reduce_mean((intersect + smooth) / (union - intersect + smooth))


def dice_coefficient(y_true, y_pred):
    numerator = 2 * tensorflow.reduce_sum(y_true * y_pred)
    denominator = tensorflow.reduce_sum(y_true + y_pred)

    return numerator / (denominator + tensorflow.keras.backend.epsilon())


In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy

model.compile(optimizer='adam',
              loss=iou_bce_loss,
              metrics=['accuracy', mean_iou])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint("model-{val_loss:.2f}.h5", monitor="val_loss", verbose=1, 
                             save_best_only=True, save_weights_only=True)

stop = EarlyStopping(monitor="val_loss", patience=2)

In [ ]:
gc.collect()
del model
gc.collect()

In [ ]:
import tensorflow as tf
his = model.fit(X_train, y_train, validation_data = (X_val, y_val), 
          epochs=20, batch_size=10, verbose=1, callbacks=[checkpoint, stop])

In [ ]:
mask_predicated = model.predict(X_val)
print(model.evaluate(X_train, y_train)) 
#[accuracy: 0.0053 - mean_iou: 0.4078]
print(model.evaluate(X_val, y_val)) 
plt.imshow(mask_predicated[1])
#[accuracy: 0.0053 - mean_iou: 0.3826]

In [ ]:
del model
gc.collect()